<center>
<img src="../../img/ods_stickers.jpg">
## Открытый курс по машинному обучению
Автор материала: Data Science Team Lead @ gojuno.com Арсений Кравченко. Материал распространяется на условиях лицензии [Creative Commons CC BY-NC-SA 4.0](https://creativecommons.org/licenses/by-nc-sa/4.0/). Можно использовать в любых целях (редактировать, поправлять и брать за основу), кроме коммерческих, но с обязательным упоминанием автора материала.

# <center>Домашнее задание № 6. Решение <br> Работа с признаками

In [1]:
import numpy as np
from demo import get_data

### Вопрос №1

При каком значении lmbda, выражение np.log(x) == scipy.stats.boxcox(x, lmbda=lmbda) будет истинным?

In [2]:
from scipy.stats import boxcox
x = np.random.rand(1)

lmbda = 0
if np.log(x) == boxcox(x, lmbda=lmbda):
    print('lmbda = {}'.format(lmbda))

lmbda = 0


### Вопрос №2

- `created.weekday() * 24 + created.hour`
- `str(created) c последующим one hot encoding`
- `created.weekday()`
- `created.hour`

Предположим, некая переменная created является признаком, содержащим datetime, это объект класса pandas.tslib.Timestamp. Какой способ извлечения признаков будет наименее полезным?

Третий и четвертый вариант достаточно популярны и имеют понятный прикладной смысл. Первый вариант на самом деле похож на сочетние третьего и четвертого признаков. А вот второй вариант довольно опасен: скорее всего, на реальных данных у многих объектов получится уникальное значение признака.

In [3]:
import json 
import pandas as pd
with open('/Users/Arseny/dev/kaggle/two_sigma_realt/train.json', 'r') as raw_data:
    data = json.load(raw_data)

created = pd.DataFrame(data).created
created = pd.to_datetime(created).apply(lambda x: str(x))

print('{} из {} значений уникальны'.format(len(created.unique()), len(created)))

48675 из 49352 значений уникальны


### Вопрос №3

Какую информацию нельзя извлечь из User-Agent? 

- Версия операционной системы
- Разрешение экрана пользователя
- Платформу, на базе которой построен браузер
- Принадлежит ли запрос роботу-индексатору Яндекса

Разрешение экрана недоступно в общем случае, хотя, как выяснили бдительные читатели, некоторые версии Internet Explorer Mobile таки включали его прямо в UserAgent.
Остальные варианты легко определяются. У некоторых возникли вопросы с роботом-индексатором Яндекса, но он обычно [честно подписан](https://yandex.com/support/webmaster/robot-workings/check-yandex-robots.xml).

### Вопрос №4

Мы решаем задачу классификации: есть пары фотографий, нужно определить, являются ли они фотографиями одного и того же объекта. Какой признак будет наиболее релевантен?

- Расстояние Левенштейна между названиями моделей фотоаппаратов, данные о модели получены из EXIF
- Евклидово расстояние между векторами, полученными из предобученной сети ResNet без полносвязных слоев
- Разница (в днях) между датами съемки фотографий, даты съемки получены из EXIF
- Разность средней яркости во всех (RGB) каналах обоих изображений

Варианты 1 и 3 - очевидная чушь. Название фотоаппарата не влияет на снимаемый объект, да и съемка могла осуществляться в любое время. Не говоря уже о том, что EXIF не всегда доступен и не всегда содержит актуальную информацию.

Вариант про яркость выглядит правдоподобным и в каких-то задачах мог бы помочь, но легко представить себе пары объектов, которые не отличить, используя только яркость (серая бетонная стена и черно-белая шахматная доска). Из предложенных вариантов только использование сети поможет определить границы объекта, по которым обычно происходит идентификация.

### Вопрос №5

Для какой из задач отбор признаков (feature selection) будет бесполезен? 

- Ускорение модели на этапе обучения
- Ускорение модели на этапе предсказания
- Борьба со слишком большими значениями признаков, ведущими к переполнению
- Повышение устойчивости модели

Сначала проверим два первых варианта:

In [4]:
from sklearn.datasets import make_classification
from sklearn.linear_model import LogisticRegression

x_data, y_data =  make_classification(n_features=3, n_informative=3, n_redundant=0, n_samples=5000)
x_data_noised = np.hstack((x_data, np.random.rand(5000, 500)))

lr, lr_n = LogisticRegression(), LogisticRegression()

In [5]:
%timeit lr.fit(x_data, y_data)

100 loops, best of 3: 5.76 ms per loop


In [6]:
%timeit lr_n.fit(x_data_noised, y_data)

1 loop, best of 3: 495 ms per loop


In [7]:
%timeit lr.predict(x_data)

The slowest run took 31.23 times longer than the fastest. This could mean that an intermediate result is being cached.
10000 loops, best of 3: 85.8 µs per loop


In [8]:
%timeit lr_n.predict(x_data_noised)

100 loops, best of 3: 2.93 ms per loop


Четвертый вариант тоже легко проверить:

In [9]:
from sklearn.model_selection import cross_val_score

In [10]:
cross_val_score(lr, x_data, y_data, scoring='neg_log_loss').mean()

-0.11898818867427889

In [11]:
cross_val_score(lr, x_data_noised, y_data, scoring='neg_log_loss').mean()

-0.2145948091912849

Для борьбы с overflow изредка может понадобиться трансформация, а не отбор.

### Вопрос №6

В каком из районов находится точка с координатами (40.825142, -73.909171)?

- Upper Manhattan
- Gangnam District
- Chinatown
- Bronx

Вопрос вызвал много вопросов типа "А можно просто посмотреть на карте?". Конечно, можно, как я могу запретить?..
Но давайте попробуем решить задачу при помощи кода.

In [12]:
from geopy.geocoders import Nominatim
nom = Nominatim()
nom.reverse((40.825142, -73.909171))

Location(962, Washington Avenue, Morrisania, Bronx, Bronx County, NYC, New York, 10456, United States of America, (40.82513605, -73.9096988274131, 0.0))

In [13]:
from geopy.geocoders import GoogleV3
gg = GoogleV3()
gg.reverse((40.825142, -73.909171))

[Location(508 E 164th St, Bronx, NY 10456, USA, (40.82527899999999, -73.908956, 0.0)),
 Location(Morrisania, Bronx, NY, USA, (40.8250663, -73.9109977, 0.0)),
 Location(West Bronx, Bronx, NY, USA, (40.8519547, -73.90070899999999, 0.0)),
 Location(Bronx, NY, USA, (40.8447819, -73.8648268, 0.0)),
 Location(New York, NY, USA, (40.7127837, -74.0059413, 0.0)),
 Location(Bronx, NY 10456, USA, (40.83102239999999, -73.9095279, 0.0)),
 Location(Bronx County, NY, USA, (40.8370495, -73.86542949999999, 0.0)),
 Location(New York-Northern New Jersey-Long Island, NY-NJ-PA, USA, (40.9590293, -74.0300122, 0.0)),
 Location(New York Metropolitan Area, USA, (40.7127761, -74.0059544, 0.0)),
 Location(New York, USA, (43.2994285, -74.21793260000001, 0.0))]

In [14]:
import reverse_geocoder as rg
rg.search((40.825142, -73.909171))

Loading formatted geocoded file...


[OrderedDict([('lat', '40.86566'),
              ('lon', '-73.9268'),
              ('name', 'Inwood'),
              ('admin1', 'New York'),
              ('admin2', 'New York County'),
              ('cc', 'US')])]

`reverse_geocoder` врет в этой ситуации, т.к. Inwood относится к Манхеттену. Так что стоит или проверить глазами на карте, или вспомнить про ансамбли из предыдущей статьи и принимать решение исходя из голосования нескольких источников данных.

### Вопрос №7

Обучите класс `CountVectorizer` из `scikit-learn`, используя колонку Description из датасета Two Sigma Connect: Rental Listing Inquires таким образом, чтобы в словаре было не более 1000 слов, не меняя прочие параметры по умолчанию. Какое слово соответствует индексу 165 в словаре обученного `CountVectorizer`?

- building
- apartment
- trust
- renthop

In [15]:
from sklearn.feature_extraction.text import CountVectorizer

with open('/Users/Arseny/dev/kaggle/two_sigma_realt/train.json', 'r') as raw_data:
    data = json.load(raw_data)

df = pd.DataFrame(data)
cv = CountVectorizer(max_features=1000)
cv.fit(df.description)

CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=1000, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip_accents=None, token_pattern='(?u)\\b\\w\\w+\\b',
        tokenizer=None, vocabulary=None)

In [16]:
# вариант с легким ковырянием в атрибутах класса
voc = cv.vocabulary_
voc = {v: k for k, v in voc.items()}
print(voc[165])


building


In [17]:
# a теперь попроще
vec = np.zeros(1000)
vec[165] = 1
print(cv.inverse_transform(vec))

[array(['building'], 
      dtype='<U16')]


### Вопрос №8

Используя датасет Two Sigma Connect: Rental Listing, обработанный при помощи [скрипта]( https://github.com/Yorko/mlcourse_open/blob/master/jupyter_notebooks/topic06_features/demo.py), отмасштабируйте его при помощи `MinMaxScaler` и удалите признаки с дисперсией меньше 0.1. Сколько признаков осталось в датасете?

- 46
- 42
- 14
- 12

In [18]:
from sklearn.preprocessing import MinMaxScaler
from sklearn.feature_selection import VarianceThreshold

df, target = get_data()
x_data = MinMaxScaler().fit_transform(df.values)

t = .1
vt = VarianceThreshold(threshold=t)
print('There are {} features with variance >= {}'.format(vt.fit_transform(x_data).shape[1], t))

There are 12 features with variance >= 0.1
